### Imports e conexão com o banco de dados

In [23]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import panel as pn
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text

pn.extension('tabulator')

# fff
# Dados de conexão - substitua com suas credenciais
DATABASE = {
    'drivername': 'postgresql',
    'host': '...',
    'port': '....',
    'username': '...',
    'password': '...',
    'database': 'LearnExpress'
}

# Criação da string de conexão
DATABASE_URL = f"{DATABASE['drivername']}://{DATABASE['username']}:{DATABASE['password']}@{DATABASE['host']}:{DATABASE['port']}/{DATABASE['database']}"

# Criação do engine
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()  # Cria uma nova sessão

### Definição dos modelos de dados

In [26]:
# Função para reiniciar a sessão
def reset_session():
    global session
    session.close()  # Fecha a sessão atual
    session = Session()  # Cria uma nova sessão

# Definição dos modelos
Base = declarative_base()

class Usuario(Base):
    __tablename__ = 'usuario'
    __table_args__ = {'schema': 'learnexpress'}

    id = Column(Integer, primary_key=True)
    nome = Column(String(50), nullable=False)
    email = Column(String(50), unique=True)
    senha = Column(String(20), unique=True)

class Curso(Base):
    __tablename__ = 'curso'
    __table_args__ = {'schema': 'learnexpress'}

    id = Column(Integer, primary_key=True)
    titulo = Column(String(50), nullable=False)
    professor = Column(String(50), nullable=False)
    descricao = Column(Text, nullable=False)
    horas_totais = Column(Integer, nullable=False)
    preco = Column(Float, nullable=False)

C:\Users\Wesley Freitas\AppData\Local\Temp\ipykernel_4852\1300317707.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Código para a tela de usuários

In [27]:
# Função para carregar dados de Usuário
def carregar_usuarios(filtro_nome=''):
    try:
        query = session.query(Usuario)
        if filtro_nome:
            query = query.filter(Usuario.nome.ilike(f'%{filtro_nome}%'))
        usuarios = query.all()
        data = [{
            'ID': user.id,
            'Nome': user.nome,
            'Email': user.email,
            'Senha': user.senha
        } for user in usuarios]
        return pd.DataFrame(data)
    except Exception as e:
        session.rollback()
        pn.state.notifications.error(f'Erro ao carregar usuários: {e}')
        return pd.DataFrame()

# Funcionalidades para Usuário
filtro_usuario = pn.widgets.TextInput(name='Filtrar por Nome', placeholder='Digite o nome do usuário')
botao_atualizar_usuario = pn.widgets.Button(name='Atualizar', button_type='primary')
botao_deletar_usuario = pn.widgets.Button(name='Deletar Usuário', button_type='danger')
botao_gerar_relatorio_usuarios_curso = pn.widgets.Button(name='Gerar Relatório de Usuários por Curso', button_type='primary')

# Atualiza tabela de Usuário
def atualizar_tabela_usuario(event):
    try:
        df = carregar_usuarios(filtro_usuario.value)
        tabela_usuario.value = df
    except Exception as e:
        session.rollback()
        pn.state.notifications.error(f'Erro ao atualizar tabela de usuários: {e}')

botao_atualizar_usuario.on_click(atualizar_tabela_usuario)

# Tabela de Usuário
tabela_usuario = pn.widgets.Tabulator(pd.DataFrame(), height=300, width=800)

# Carregar todos os usuários inicialmente
atualizar_tabela_usuario(None)  

# Formulário para adicionar Usuário
nome_input = pn.widgets.TextInput(name='Nome')
email_input = pn.widgets.TextInput(name='Email')
senha_input = pn.widgets.PasswordInput(name='Senha')
botao_adicionar_usuario = pn.widgets.Button(name='Adicionar Usuário', button_type='success')

def adicionar_usuario(event):
    try:
        if nome_input.value and email_input.value and senha_input.value:
            usuario_existente = session.query(Usuario).filter_by(email=email_input.value).first()
            if usuario_existente:
                print("Este e-mail já está cadastrado.")
                return
            
            novo_usuario = Usuario(nome=nome_input.value, email=email_input.value, senha=senha_input.value)
            session.add(novo_usuario)
            session.commit()
            print("Usuário adicionado com sucesso.")
            
            # Limpa os campos após adicionar
            nome_input.value = ""
            email_input.value = ""
            senha_input.value = ""
    except Exception as e:
        print(f"Erro ao adicionar usuário: {e}")

botao_adicionar_usuario.on_click(adicionar_usuario)

# Campo de texto para inserir o ID do usuário a ser deletado
usuario_id_input = pn.widgets.IntInput(name='ID do Usuário', value=None)

def deletar_usuario(event):
    usuario_id = usuario_id_input.value
    if usuario_id is not None:
        try:
            usuario_para_deletar = session.query(Usuario).get(usuario_id)
            
            if usuario_para_deletar:
                session.delete(usuario_para_deletar)
                session.commit()
                if pn.state.notifications:
                    pn.state.notifications.success(f'Usuário com ID {usuario_id} deletado com sucesso!')
                atualizar_tabela_usuario(None)  # Atualiza a tabela após a exclusão
                usuario_id_input.value = None  # Limpar o campo após a exclusão
            else:
                if pn.state.notifications:
                    pn.state.notifications.error('Usuário não encontrado.')
        except Exception as e:
            session.rollback()
            if pn.state.notifications:
                pn.state.notifications.error(f'Erro ao deletar usuário: {e}')
            else:
                print(f'Erro ao deletar usuário: {e}')
    else:
        if pn.state.notifications:
            pn.state.notifications.warning('Por favor, insira um ID válido.')

botao_deletar_usuario.on_click(deletar_usuario)

def gerar_relatorio_usuarios_por_curso():
    try:
        query = text('''
            SELECT c.titulo AS titulo_curso, u.nome AS nome_usuario, u.email AS email_usuario
            FROM learnexpress.curso c
            JOIN learnexpress.pedido p ON c.id = p.idCurso
            JOIN learnexpress.usuario u ON p.idUsuario = u.id
            ORDER BY c.titulo;
        ''')
        
        resultado = session.execute(query).fetchall()

        data = [{
            'Título do Curso': row[0], 
            'Nome do Usuário': row[1], 
            'Email do Usuário': row[2] 
        } for row in resultado]
        
        df_relatorio = pd.DataFrame(data)
        return df_relatorio

    except Exception as e:
        session.rollback()
        if pn.state.notifications:
            pn.state.notifications.error(f'Erro ao gerar relatório: {e}')
        else:
            print(f'Erro ao gerar relatório: {e}')
        return pd.DataFrame()

# Tabela para exibir o relatório
tabela_relatorio_usuarios_curso = pn.widgets.Tabulator(pd.DataFrame(), height=300, width=800)

# Função para gerar e exibir o relatório
def atualizar_relatorio_usuarios_curso(event):
    try:
        df_relatorio = gerar_relatorio_usuarios_por_curso()
        tabela_relatorio_usuarios_curso.value = df_relatorio
    except Exception as e:
        pn.state.notifications.error(f'Erro ao atualizar relatório: {e}')

# Conectar o botão à função de atualização
botao_gerar_relatorio_usuarios_curso.on_click(atualizar_relatorio_usuarios_curso)

form_usuario = pn.Column(
    pn.pane.Markdown("### Adicionar Novo Usuário"),
    nome_input,
    email_input,
    senha_input,
    botao_adicionar_usuario,
    pn.pane.Markdown("### Deletar Usuário"),
)

# Tela de Usuário
layout_usuario = pn.Column(
    pn.pane.Markdown("## Gerenciamento de Usuários"),
    filtro_usuario,
    botao_atualizar_usuario,
    tabela_usuario,
    form_usuario,
    botao_deletar_usuario,
    usuario_id_input,
     pn.pane.Markdown("## Relatório de Usuários Cadastrados por Curso"),
    botao_gerar_relatorio_usuarios_curso,
    tabela_relatorio_usuarios_curso
)


### Código para a tela de cursos

In [28]:
# Função para carregar dados de Curso
def carregar_cursos(filtro_titulo=''):
    try:
        query = session.query(Curso)
        if filtro_titulo:
            query = query.filter(Curso.titulo.ilike(f'%{filtro_titulo}%'))
        cursos = query.all()
        data = [{
            'ID': curso.id,
            'Título': curso.titulo,
            'Professor': curso.professor,
            'Descrição': curso.descricao,
            'Horas Totais': curso.horas_totais,
            'Preço': curso.preco
        } for curso in cursos]
        return pd.DataFrame(data)
    except Exception as e:
        session.rollback()
        pn.state.notifications.error(f'Erro ao carregar cursos: {e}')
        return pd.DataFrame()

# Widgets para Curso
filtro_curso = pn.widgets.TextInput(name='Filtrar por Título', placeholder='Digite o título do curso')
botao_atualizar_curso = pn.widgets.Button(name='Atualizar', button_type='primary')
botao_deletar_curso = pn.widgets.Button(name='Deletar Curso', button_type='danger')
botao_relatorio = pn.widgets.Button(name='Gerar Relatório de Compras', button_type='primary')

# Atualiza a tabela de Curso
def atualizar_tabela_curso(event):
    try:
        df = carregar_cursos(filtro_curso.value)
        tabela_curso.value = df
    except Exception as e:
        session.rollback()
        pn.state.notifications.error(f'Erro ao atualizar tabela de cursos: {e}')

botao_atualizar_curso.on_click(atualizar_tabela_curso)

# Tabela de Curso
tabela_curso = pn.widgets.Tabulator(pd.DataFrame(), height=300, width=800)

# Carregar todos os cursos inicialmente
atualizar_tabela_curso(None) 

# Formulário para adicionar Curso
titulo_input = pn.widgets.TextInput(name='Título')
professor_input = pn.widgets.TextInput(name='Professor')
descricao_input = pn.widgets.TextAreaInput(name='Descrição')
horas_totais_input = pn.widgets.IntInput(name='Horas Totais', value=1)
preco_input = pn.widgets.FloatInput(name='Preço', value=0.0)
botao_adicionar_curso = pn.widgets.Button(name='Adicionar Curso', button_type='success')

def adicionar_curso(event):
    if titulo_input.value and professor_input.value and descricao_input.value and horas_totais_input.value and preco_input.value is not None:
        novo_curso = Curso(
            titulo=titulo_input.value,
            professor=professor_input.value,
            descricao=descricao_input.value,
            horas_totais=horas_totais_input.value,
            preco=preco_input.value
        )
        session.add(novo_curso)
        try:
            session.commit()
            if pn.state.notifications:
                pn.state.notifications.success('Curso adicionado com sucesso!')
            atualizar_tabela_curso(None)
            titulo_input.value = ''
            professor_input.value = ''
            descricao_input.value = ''
            horas_totais_input.value = 1
            preco_input.value = 0.0
        except Exception as e:
            session.rollback()
            if pn.state.notifications:
                pn.state.notifications.error(f'Erro ao adicionar curso: {e}')
            else:
                print(f'Erro ao adicionar curso: {e}')
    else:
        if pn.state.notifications:
            pn.state.notifications.warning('Por favor, preencha todos os campos.')
        else:
            print('Por favor, preencha todos os campos.')


botao_adicionar_curso.on_click(adicionar_curso)

# Campo de texto para inserir o ID do curso a ser deletado
curso_id_input = pn.widgets.IntInput(name='ID do Curso', value=None)

def deletar_curso(event):
    curso_id = curso_id_input.value
    if curso_id is not None:
        try:
            curso_para_deletar = session.query(Curso).get(curso_id)
            
            if curso_para_deletar:
                session.delete(curso_para_deletar)
                session.commit()
                if pn.state.notifications:
                    pn.state.notifications.success(f'Curso com ID {curso_id} deletado com sucesso!')
                atualizar_tabela_curso(None)  # Atualiza a tabela após a exclusão
                curso_id_input.value = None  # Limpar o campo após a exclusão
            else:
                if pn.state.notifications:
                    pn.state.notifications.error('Curso não encontrado.')
        except Exception as e:
            session.rollback()
            if pn.state.notifications:
                pn.state.notifications.error(f'Erro ao deletar curso: {e}')
            else:
                print(f'Erro ao deletar curso: {e}')
    else:
        if pn.state.notifications:
            pn.state.notifications.warning('Por favor, insira um ID válido.')

botao_deletar_curso.on_click(deletar_curso)

from sqlalchemy import text

def gerar_relatorio_compras():
    try:
        query = text('''
            SELECT c.titulo AS titulo_curso, COUNT(p.id) AS total_compras
            FROM learnexpress.curso c
            LEFT JOIN learnexpress.pedido p ON c.id = p.idCurso
            GROUP BY c.titulo
            ORDER BY total_compras DESC;
        ''')
        
        resultado = session.execute(query).fetchall()

        data = [{
            'Título do Curso': row[0],  
            'Total de Compras': row[1]  
        } for row in resultado]
        
        df_relatorio = pd.DataFrame(data)
        return df_relatorio

    except Exception as e:
        session.rollback()
        if pn.state.notifications:
            pn.state.notifications.error(f'Erro ao gerar relatório: {e}')
        else:
            print(f'Erro ao gerar relatório: {e}')
        return pd.DataFrame()

# Tabela para exibir o relatório
tabela_relatorio = pn.widgets.Tabulator(pd.DataFrame(), height=300, width=800)

# Função para atualizar o relatório na interface
def atualizar_relatorio(event):
    df_relatorio = gerar_relatorio_compras()
    tabela_relatorio.value = df_relatorio

botao_relatorio.on_click(atualizar_relatorio)

form_curso = pn.Column(
    pn.pane.Markdown("### Adicionar Novo Curso"),
    titulo_input,
    professor_input,
    descricao_input,
    horas_totais_input,
    preco_input,
    botao_adicionar_curso,
    pn.pane.Markdown("### Deletar Curso"),
)

# Tela de Curso
layout_curso = pn.Column(
    pn.pane.Markdown("## Gerenciamento de Cursos"),
    filtro_curso,
    botao_atualizar_curso,
    tabela_curso,
    form_curso,
    curso_id_input,
    botao_deletar_curso,
    
    pn.layout.Divider(),
    pn.pane.Markdown("### Relatório de Compras por Curso"),
    botao_relatorio,
    tabela_relatorio
)


### Abas alternativas

In [29]:


# Abas para alternar entre Usuário e Curso
abas = pn.Tabs(
    ('Usuários', layout_usuario),
    ('Cursos', layout_curso)
)

# Servir a aplicação com as abas
abas.servable(title="Gerenciamento de Usuários e Cursos")

BokehModel(combine_events=True, render_bundle={'docs_json': {'b957e3d7-a99e-47b9-8834-3c68a5d05b41': {'version…